<a href="https://colab.research.google.com/github/RedRev86/Python-Data-Science-Alura/blob/main/OCR_Texto_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Instalación de dependencias
!pip install pdf2image opencv-python
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [13]:
# Subir PDF
from google.colab import files

def upload_pdf():
    uploaded = files.upload()
    for filename in uploaded:
        if filename.lower().endswith(".pdf"):
            print(f"PDF cargado: {filename}")
            return filename
    raise ValueError("No se encontró un archivo PDF.")

In [14]:
# Clase y funciones auxiliares
import os, json, shutil, requests, cv2
import numpy as np
from pdf2image import convert_from_path

class DocumentAPIProcessor:
    def __init__(self, api_url: str = "https://cc-hts-ai-cc-document-scanner-qwen2.ydkf28.easypanel.host/analyze_image"):
        self.api_url = api_url

    def pdf_to_images(self, pdf_path: str, output_dir: str, dpi: int = 300, target_width: int = None, target_height: int = None):
        os.makedirs(output_dir, exist_ok=True)
        pages = convert_from_path(pdf_path, dpi=dpi)
        image_paths = []
        for i, page in enumerate(pages):
            img = cv2.cvtColor(np.array(page), cv2.COLOR_RGB2BGR)
            if target_width or target_height:
                h, w = img.shape[:2]
                if target_width and not target_height:
                    scale = target_width / w
                    target_height = int(h * scale)
                elif target_height and not target_width:
                    scale = target_height / h
                    target_width = int(w * scale)
                img = cv2.resize(img, (target_width, target_height), interpolation=cv2.INTER_AREA)
            path = os.path.join(output_dir, f"page_{i+1}.jpg")
            cv2.imwrite(path, img)
            image_paths.append(path)
        return image_paths

    def process_image(self, image_path: str, prompt: str):
        with open(image_path, 'rb') as f:
            files_payload = {'file': f}
            data = {'prompt': prompt}
            response = requests.post(self.api_url, files=files_payload, data=data)
            response.raise_for_status()
            return response.json()

    def process_document(self, image_path: str):
        prompt = """
Extrae del documento los siguientes elementos y devuélvelos en formato JSON estructurado:
{
  "titulo_poema": "",
  "autor": "",
  "referencia": "",
  "texto_extraido_completo": "",
  "estrofas": [
    {
      "versos": ["", "", ""]
    }
  ]
}
Instrucciones:
- Si el título del poema aparece, colócalo en \"titulo_poema\".
- Si el autor aparece, inclúyelo en \"autor\".
- Si hay referencias editoriales, fecha o colección, colócalas en \"referencia\".
- En \"texto_extraido_completo\", incluye todo el texto del poema con saltos de línea conservados.
- Separa cada estrofa del poema en el campo \"estrofas\", y divide cada verso en una lista dentro de cada estrofa.
"""
        return self.process_image(image_path, prompt)

def clean_nested_json(input_data):
    try:
        if 'result' not in input_data:
            return None
        raw = input_data['result'].replace('```json', '').replace('```', '').strip()
        return json.loads(raw)
    except Exception as e:
        print("Error limpiando JSON:", e)
        return None

In [15]:
# Procesamiento del PDF
pdf_filename = upload_pdf()
output_dir = "temp_images"
TARGET_WIDTH = 780

processor = DocumentAPIProcessor()
image_paths = processor.pdf_to_images(pdf_filename, output_dir, target_width=TARGET_WIDTH)

Saving Poesía erotica Díaz Miron_p1-3.pdf to Poesía erotica Díaz Miron_p1-3.pdf
PDF cargado: Poesía erotica Díaz Miron_p1-3.pdf


In [16]:
# Llamada a la API y limpieza del resultado
api_results = [processor.process_document(path) for path in image_paths]

#cleaned_results = {
 #   "titulo_poema": "",
 #   "autor": "",
 #   "referencia": "",
 #   "texto_extraido_completo": "",
 #   "estrofas": []
 #                }

#for i, result in enumerate(api_results):
 #   cleaned = clean_nested_json(result)
 #   if cleaned:
 #       cleaned_results[f"pagina_{i+1}"] = cleaned


# Versión con almacenamiento por página
cleaned_results = {}
for i, result in enumerate(api_results):
    cleaned = clean_nested_json(result)
    if cleaned:
        cleaned_results[f"pagina_{i+1}"] = cleaned

# Si aplicas el script anterior debes quitar lo siguiente
#for result in api_results:
 #   cleaned = clean_nested_json(result)
 #   if cleaned:
 #       if cleaned.get("titulo_poema"):
 #           cleaned_results["titulo_poema"] = cleaned["titulo_poema"]
 #       if cleaned.get("autor"):
 #           cleaned_results["autor"] = cleaned["autor"]
 #       if cleaned.get("referencia"):
 #           cleaned_results["referencia"] = cleaned["referencia"]
 #       if cleaned.get("texto_extraido_completo"):
 #           cleaned_results["texto_extraido_completo"] += cleaned["texto_extraido_completo"] + "\n"
 #       if cleaned.get("estrofas"):
 #           cleaned_results["estrofas"].extend(cleaned["estrofas"])

In [17]:
# Visualización y limpieza de archivos
print(json.dumps(cleaned_results, indent=2, ensure_ascii=False))

shutil.rmtree(output_dir)
os.remove(pdf_filename)

{
  "pagina_1": {
    "titulo_poema": "VIGILIA Y SUEÑO",
    "autor": "Salvador Díaz Miron",
    "referencia": "FCE. Méx, 1979",
    "texto_extraido_completo": "La moza lucha con el mancebo\n-su prometido y hermoso efebo-\ny vence a costa de un traje nuevo.\n\nY huye sin mancha ni deterioro\nen la pureza y en el decoro,\ny es un gran lirio de nieve y oro.\n\nY entre la sombra solemne y bruna,\nyerra en el mate jardín, cual una\nvisión compuesta de aroma y luna.\n\nY gana el cuarto, y ante un espejo,\ny con orgullo de amargo dejo,\ncambia sonrisas con un reflejo.",
    "estrofas": [
      {
        "versos": [
          "La moza lucha con el mancebo",
          "-su prometido y hermoso efebo-",
          "y vence a costa de un traje nuevo."
        ]
      },
      {
        "versos": [
          "Y huye sin mancha ni deterioro",
          "en la pureza y en el decoro,",
          "y es un gran lirio de nieve y oro."
        ]
      },
      {
        "versos": [
          "Y entre la s

Es el mismo código que OCR_Poemas_Extractor , pero se agregó la siguiente celda; sin embargo ya es posible ver el texto extraído.

In [7]:
# Visualización del texto extraído. NO FUNCIONA PORQUE NO ESTÁ DEFINIDA LA VARIABLE:
if extracted_data_str:
    from IPython.display import display, Markdown
    # Convierte los \n en saltos de línea reales
    texto_formateado = extracted_data_str.replace("\\n", "\n")
    print("Texto extraído:")
    display(Markdown(f"```text\n{texto_formateado}\n```"))
else:
    print("No hay texto extraído para mostrar.")


NameError: name 'extracted_data_str' is not defined